In [1]:
# Import packages
import os

import numpy as np
import pandas as pd

import torch
from torch import nn

In [ ]:
# Define constants
valid_share = 0.4
torch.manual_seed(9999)
np.random.seed(9998)

In [ ]:
# Specify directory/file paths
data_dir = "Data"

sim_data_file = "DIBS_simulated_fmri_var4model.csv"

In [ ]:
# Check that CUDA GPUs are available
if torch.cuda.is_available():
    device = torch.cuda.current_device()
    print(f'Number of GPUs available: {torch.cuda.device_count()}')
    for i in range(torch.cuda.device_count()):
        print(f'GPU {i+1}: {torch.cuda.get_device_name(i)}')
else:
    device = torch.device("cpu")
    print("GPU not available--using CPU instead.")

In [ ]:
# Load data
data = pd.read_csv(os.path.join(data_dir, sim_data_file), header=0)
data.head()

In [ ]:
# Shuffle data and create test dataset
shuffled_idx = np.random.permutation(len(data))
data = data.loc[shuffled_idx].reset_index(drop=True)

train_data = data.loc[0:int(len(data) * (1-valid_share)), ]
train_data.shape

test_idx = [idx for idx in data.index if idx not in train_data.index]
test_data = data.loc[test_idx, ]
print("Train data: ", train_data.shape, "\nTest data: ", test_data.shape)

In [ ]:
# Create input and target sequences for training data
train_input_seq = []
train_target_seq = []

seq_len = 100
for obs in range(len(train_data)):
    obs_data = train_data.iloc[obs]
    for i in range(len(obs_data) - seq_len):
        train_input_seq.append(obs_data[i:i+seq_len].values)
        train_target_seq.append(obs_data[i+seq_len])
        i += 1
print("# train input seqs: ", len(train_input_seq), "\n# train target seqs: ", len(train_target_seq))

In [ ]:
# Convert sequences to PyTorch tensors
train_input_tensor = torch.from_numpy(np.array(train_input_seq)).float().unsqueeze(-1)
train_input_tensor = train_input_tensor.to(device) # Note that "to" method for tensors is not in-place operation (like it is for nn.Module/model below)
train_input_tensor.shape

In [ ]:
train_target_tensor = torch.from_numpy(np.array(train_target_seq)).float().unsqueeze(-1)
train_target_tensor = train_target_tensor.to(device) # Note that "to" method for tensors is not in-place operation (like it is for nn.Module/model below)
train_target_tensor.shape